# Viewing S3 Bucket Granules with S3FS
### Author: Chris Battisto
### Date Authored: 1-26-22

### Timing

Exercise: 5 minutes

<p></p>

<div style="background:#fc9090;border:1px solid #cccccc;padding:5px 10px;"><big><b>Note:  </b>This notebook <em><strong>will only run in an environment with <a href="https://disc.gsfc.nasa.gov/information/glossary?keywords=%22earthdata%20cloud%22&amp;title=AWS%20region">us-west-2 AWS access</a></strong></em>.</big></div>

### Overview

This notebook demonstrates viewing granules inside the GES DISC S3 bucket through the S3FS Python library.

### Prerequisites

This notebook was written using Python 3.8, and requires these libraries and files: 

- Xarray
- S3FS
  - S3FS documentation: https://s3fs.readthedocs.io/en/latest/install.html
- netrc file with valid Earthdata Login credentials.
- Approval to access the GES DISC archives with your Earthdata credentials (https://disc.gsfc.nasa.gov/earthdata-login)


### Import Libraries

In [1]:
import s3fs
import xarray as xr
import requests

### Get S3 Credentials and Mount the S3 Filesystem

In [2]:
gesdisc_s3 = "https://data.gesdisc.earthdata.nasa.gov/s3credentials"
response = requests.get(gesdisc_s3).json() 

fs = s3fs.S3FileSystem(key=response['accessKeyId'],
                    secret=response['secretAccessKey'],
                    token=response['sessionToken'],
                    client_kwargs={'region_name':'us-west-2'})

# Check that the file system is intact as an S3FileSystem object, which means that token is valid
# Common causes of rejected S3 access tokens include incorrect passwords stored in the netrc file, or a non-existent netrc file
type(fs)

s3fs.core.S3FileSystem

### Explore S3FS Bucket Contents

Once you have your S3 URLs and the filesystem mounted, you can access and view information as if it were stored locally.

In [3]:
# You can use filesystem commands like ls and glob to view contents:

print('Current datasets: ')
fs.glob('s3://gesdisc-cumulus-prod-protected/')

Current datasets: 


['gesdisc-cumulus-prod-protected/.Zxt5FGhb5T',
 'gesdisc-cumulus-prod-protected/Aqua_AIRS_Level1',
 'gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level1',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level2',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level3',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level2',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level2G',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level3',
 'gesdisc-cumulus-prod-protected/CHIRP',
 'gesdisc-cumulus-prod-protected/CMS',
 'gesdisc-cumulus-prod-protected/FLDAS',
 'gesdisc-cumulus-prod-protected/GLDAS',
 'gesdisc-cumulus-prod-protected/GOSAT_TANSO_Level2',
 'gesdisc-cumulus-prod-protected/GPM_L1C',
 'gesdisc-cumulus-prod-protected/GPM_L2',
 'gesdisc-cumulus-prod-protected/GPM_L3',
 'gesdisc-cumulus-prod-protected/GRACEDA',
 'gesdisc-cumulus-prod-protected/JPSS1_Sounder_Level2',
 'gesdisc-cumulus-prod-protected/Landslide',
 'gesdisc-cumulus-prod-protected/M2T1NXSLV',
 'gesdi

In [4]:
fs.ls('s3://gesdisc-cumulus-prod-protected/')

['gesdisc-cumulus-prod-protected/.Zxt5FGhb5T',
 'gesdisc-cumulus-prod-protected/Aqua_AIRS_Level1',
 'gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level1',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level2',
 'gesdisc-cumulus-prod-protected/Aura_MLS_Level3',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level2',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level2G',
 'gesdisc-cumulus-prod-protected/Aura_OMI_Level3',
 'gesdisc-cumulus-prod-protected/CHIRP',
 'gesdisc-cumulus-prod-protected/CMS',
 'gesdisc-cumulus-prod-protected/FLDAS',
 'gesdisc-cumulus-prod-protected/GLDAS',
 'gesdisc-cumulus-prod-protected/GOSAT_TANSO_Level2',
 'gesdisc-cumulus-prod-protected/GPM_L1C',
 'gesdisc-cumulus-prod-protected/GPM_L2',
 'gesdisc-cumulus-prod-protected/GPM_L3',
 'gesdisc-cumulus-prod-protected/GRACEDA',
 'gesdisc-cumulus-prod-protected/JPSS1_Sounder_Level2',
 'gesdisc-cumulus-prod-protected/Landslide',
 'gesdisc-cumulus-prod-protected/M2T1NXSLV',
 'gesdi

### Open a Granule Directly in Xarray

Individual granule metadata can be retrived using S3FS, and can be opened with Xarray natively.

In [5]:
#fn = 's3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2019/03/MERRA2_400.tavg1_2d_slv_Nx.20190313.nc4'
fn = 's3://gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2005/001/SNDR.AQUA.AIRS_IM.20050101T0005.m06.g001.L2_CLIMCAPS_RET.std.v02_39.G.210522010807.nc'

fs.info(fn)

{'ETag': '"e3ca7c27dab27299ff44ec23da303cb4-1"',
 'LastModified': datetime.datetime(2022, 2, 16, 13, 59, 57, tzinfo=tzutc()),
 'size': 22012857,
 'name': 'gesdisc-cumulus-prod-protected/Aqua_Sounder_Level2/SNDRAQIML2CCPRET.2/2005/001/SNDR.AQUA.AIRS_IM.20050101T0005.m06.g001.L2_CLIMCAPS_RET.std.v02_39.G.210522010807.nc',
 'type': 'file',
 'StorageClass': 'STANDARD',
 'VersionId': None,
 'ContentType': 'application/x-netcdf'}

Open the file by passing <code>fs.open(fn)</code> into <code>xr.open_dataset()</code>

In [6]:
ds = xr.open_dataset(fs.open(fn),
                     decode_cf=True,)

ds

<xarray.Dataset>
Dimensions:                    (atrack: 45, xtrack: 30, fov: 9, utc_tuple: 8,
                                fov_poly: 8, spatial: 3, attitude: 3,
                                air_pres: 100, air_pres_h2o: 66,
                                air_pres_lay: 100, surf_wnum_ir: 100,
                                surf_freq_mw: 7, cld_lay: 2, bnds_1d: 2)
Coordinates: (12/15)
    lat                        (atrack, xtrack) float32 ...
    fov_lat                    (atrack, xtrack, fov) float32 ...
    lon                        (atrack, xtrack) float32 ...
    fov_lon                    (atrack, xtrack, fov) float32 ...
    subsat_lat                 (atrack) float32 ...
    subsat_lon                 (atrack) float32 ...
    ...                         ...
    surf_ir_wnum               (atrack, xtrack, surf_wnum_ir) float32 ...
  * air_pres                   (air_pres) float32 1.606 3.838 ... 1.1e+05
  * air_pres_h2o               (air_pres_h2o) float32 5.153e+03 ... 1.1e+05
  * air_pres_lay               (air_pres_lay) float32 0.2837 2.486 ... 1.085e+05
    cld_lay_lbl                (cld_lay) object ...
  * surf_freq_mw               (surf_freq_mw) float32 2.38e+10 ... 1.833e+11
Dimensions without coordinates: atrack, xtrack, fov, utc_tuple, fov_poly,
                                spatial, attitude, surf_wnum_ir, cld_lay,
                                bnds_1d
Data variables: (12/144)
    obs_id                     (atrack, xtrack) object ...
    fov_obs_id                 (atrack, xtrack, fov) object ...
    obs_time_tai93             (atrack, xtrack) datetime64[ns] ...
    obs_time_utc               (atrack, xtrack, utc_tuple) float32 ...
    lat_geoid                  (atrack, xtrack) float32 ...
    lon_geoid                  (atrack, xtrack) float32 ...
    ...                         ...
    ir_precip_est_24hr_qc      (atrack, xtrack, fov) float32 ...
    air_pres_nsurf             (atrack, xtrack) float32 ...
    air_pres_h2o_nsurf         (atrack, xtrack) float32 ...
    air_pres_lay_nsurf         (atrack, xtrack) float32 ...
    air_pres_lay_bnds          (air_pres_lay, bnds_1d) float32 ...
    mw_surf_class              (atrack, xtrack) float32 ...
Attributes: (12/82)
    keywords:                          ATMOSPHERE > ATMOSPHERIC TEMPERATURE >...
    Conventions:                       CF-1.6, ACDD-1.3
    source:                            AIRS and AMSU-A instrument telemetry
    processing_level:                  2
    product_name_type_id:              L2_CLIMCAPS_RET
    comment:                           
    ...                                ...
    AutomaticQualityFlag:              Passed
    qa_pct_data_missing:               0.0
    qa_pct_data_geo:                   100.0
    qa_pct_data_sci_mode:              100.0
    qa_no_data:                        FALSE
    metadata_link:                     http://disc.sci.gsfc.nasa.gov/